In [1]:
#Implementing a hybrid network, that looks at the STFT of the signal, and the time domain version of the signal
#Combines and tries to classify as being an accent or not.
import glob, os
import scipy.io, scipy.io.wavfile
import pandas as pd
%matplotlib inline

import pickle
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt

num_neurons = 5000


files = glob.glob("../sound_data/*.wav")
data_keys = ['filename', 'time_data', 'stft_data', 'stft_time_axis', 'Zxx', 'label']
list_of_dicts = []
labels = []
for file in files:
    fs, data = scipy.io.wavfile.read(file)
    filename = os.path.basename(file)
    data = scipy.signal.resample(data, num_neurons)
    #Does zero padding automatically apparently
    stft, t, Zxx = signal.stft(data, fs, nperseg=5000)
    label = 1
    if 'english' in filename:
        label = 0
    labels.append(label)
    data_values = [filename, data, stft, t, Zxx, label]
    data_dict = dict(zip(data_keys, data_values))
    list_of_dicts.append(data_dict)
    #print(len(data))
    print(len(stft))
    #plt.plot(range(len(data)) , data)
    


2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501
2501


KeyboardInterrupt: 

In [3]:
#Separate data for use in hybrid network(s)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
test_size = 0.15


X_train, X_test, y_train, y_test = train_test_split(list_of_dicts, labels, test_size = test_size, random_state = 32)


time_train = []
stft_train = []

for index in range(len(X_train)):
    time_data = X_train[index]['time_data']
    stft_data = X_train[index]['stft_data']
    time_train.append(time_data)
    stft_train.append(stft_data)
    
time_test = []
stft_test = []
    

for index in range(len(X_test)):
    time_data = X_test[index]['time_data']
    stft_data = X_test[index]['stft_data']
    time_test.append(time_data)
    stft_test.append(stft_data)
    
    
print('num_objs T: ' + str(len(time_test)) + 'obj_len T: ' + str(len(time_test[0])))
print('num_objs F: ' + str(len(stft_test)) + 'obj_len F: ' + str(len(stft_test[0])))

num_objs T: 22obj_len T: 5000
num_objs F: 22obj_len F: 2501


In [12]:
#Implementation of hybrid network using Keras. 
#Implementation 1: Two different networks, come together in an (alpha) + (1-alpha) weighting
from keras import optimizers
from keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, concatenate
from sklearn.preprocessing import OneHotEncoder
#Shapes for inputs
inputTime = Input(shape=(num_neurons,))
inputFFT = Input(shape=(len(stft_test[0]),))

#Time domain branch
#First layer should be 5000 neurons
x = Dense(num_neurons, activation="relu")(inputTime)
x = Dense(1250, activation="relu")(x)
x = Dense(125, activation="relu")(x)
x = Model(inputs=inputTime, outputs=x)

#FFT branch
y = Dense(num_neurons, activation="relu")(inputFFT)
y = Dense(1250, activation="relu")(y)
y = Dense(125, activation="relu")(y)
y = Model(inputs=inputFFT, outputs=y)

#Combine together
combined = concatenate([x.output, y.output])

# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(1, activation="relu")(combined)
z = Dense(1, activation="linear")(z)
#This will output a binary (does he have a foreign accent? or not?)
z = Dense(1, activation="softmax")(z)

# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input], outputs=z)

In [15]:
#Compile model and run
Adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy', optimizer=Adam, metrics=['accuracy'])
training_data = [time_train, stft_train]
test_data = [time_test, stft_test]
print('Keras Model Starting...')

Keras Model Starting...


In [ ]:
#Run model
history=model.fit(training_data, y_train , validation_data=(test_data, y_test), epochs=1000)

Train on 123 samples, validate on 22 samples
Epoch 1/1000
123/123 [==============================] - 3s 21ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 2/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 3/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 4/1000
123/123 [==============================] - 3s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 5/1000
123/123 [==============================] - 3s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 6/1000
123/123 [==============================] - 3s 21ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 7/1000
123/123 [==============================] - 3s 23ms/step - loss: 8.7262 -

Epoch 57/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 58/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 59/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 60/1000
123/123 [==============================] - 3s 21ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 61/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 62/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 63/1000
123/123 [==============================] - 3s 22ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 -

123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 114/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 115/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 116/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 117/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 118/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 119/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_acc

Epoch 169/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 170/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 171/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 172/1000
123/123 [==============================] - 3s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 173/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 174/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 175/1000
123/123 [==============================] - 3s 22ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6

Epoch 225/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 226/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 227/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 228/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 229/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 230/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 231/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6

...

Epoch 449/1000
123/123 [==============================] - 3s 22ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 450/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 451/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 452/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 453/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 454/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 455/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6

Epoch 505/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 506/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 507/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 508/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 509/1000
123/123 [==============================] - 3s 21ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 510/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 511/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6

Epoch 561/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 562/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 563/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 564/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 565/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 566/1000
123/123 [==============================] - 3s 21ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 567/1000
123/123 [==============================] - 3s 23ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6

Epoch 617/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 618/1000
123/123 [==============================] - 2s 19ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 619/1000
123/123 [==============================] - 3s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 620/1000
123/123 [==============================] - 2s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 621/1000
123/123 [==============================] - 3s 21ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 622/1000
123/123 [==============================] - 3s 20ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 623/1000
123/123 [==============================] - 3s 21ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6

Epoch 673/1000
123/123 [==============================] - 3s 22ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 674/1000
123/123 [==============================] - 3s 23ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 675/1000
123/123 [==============================] - 3s 23ms/step - loss: 8.7262 - accuracy: 0.4309 - val_loss: 6.9697 - val_accuracy: 0.5455
Epoch 676/1000
 64/123 [==============>...............] - ETA: 1s - loss: 8.8645 - accuracy: 0.4219 